## Imports

In [1]:
%matplotlib inline

import psycopg2
import pandas as pd
import numpy as np
import re
import pylab as pl
import datetime

## Get data from the database

In [2]:
conn = psycopg2.connect(database="training_2015", user="maharishi", password="maharishi", host="dssgsummer2014postgres.c5faqozfo86k.us-west-2.rds.amazonaws.com", port="5432")

data = pd.read_sql("select (CASE WHEN \"INSPECTION STATUS\"= 'PASSED' THEN 1\
            WHEN \"INSPECTION STATUS\"='FAILED' THEN 0 \
       END) as actual_class,* \
       from esha_m.building_info \
        where \"INSPECTION STATUS\"='PASSED' or \"INSPECTION STATUS\"='FAILED' order by  \"VIOLATION DATE\"  limit 1000", conn)


## Clean and standardize the data

In [3]:
def space_to_snake(column_name):
    """Replace space with underscore """
    tmp = re.sub(r"\s+",'_',column_name)
    return tmp.lower()

tmp = []
for col in data.columns:
    tmp.append(space_to_snake(col))
data.columns=tmp

data.violation_date = pd.to_datetime(data.violation_date)
data.violation_status_date = pd.to_datetime(data.violation_status_date)
data.violation_last_modified_date = pd.to_datetime(data.violation_last_modified_date)

##View the data

In [13]:
data.head(1)

,actual_class,violation_last_modified_date,violation_date,violation_code,violation_status,violation_status_date,violation_description,violation_location,violation_inspector_comments,violation_ordinance,...,estimated_cost,amount_waived,amount_paid,total_fee,suffix,work_description,latitude,longitude,location,address
0,0,2006-02-22,2006-01-02,RF302021,OPEN,NaT,PRV RECEIVER,None,"FOR ONE 2-1/2 TON COPELAND COMPRESSOR, LOCATED...",Install safety valve in liquid receiver or hig...,...,10000.00,0.00,608.00,608.00,None,SOUTHWEST CORNER OF PROPERTY - EXISTING DOUBLE...,41.976529,-87.659724,"(41.97652938043876, -87.65972361874651)",5205 N BROADWAY


In [14]:
data.dtypes

actual_class                             int64
violation_last_modified_date    datetime64[ns]
violation_date                  datetime64[ns]
violation_code                          object
violation_status                        object
violation_status_date           datetime64[ns]
violation_description                   object
violation_location                      object
violation_inspector_comments            object
violation_ordinance                     object
inspector_id                            object
inspection_number                        int64
inspection_status                       object
inspection_waived                         bool
inspection_category                     object
department_bureau                       object
property_group                           int64
ssa                                    float64
permit#                                 object
permit_type                             object
estimated_cost                          object
amount_waived

## Generate features

In [16]:
def dayspassed(input_date):
    now = datetime.datetime.now()
    return (now-input_date).days

data['dayspassed']=data.violation_date.apply(lambda x: dayspassed(x))

## Transform the data to use with sci-kit

In [19]:
selected_features = ['dayspassed','inspection_category','department_bureau','property_group','permit_type','total_fee' ]

In [20]:
train_data=data[0:99]

In [21]:
test_data=data[100:199]

In [22]:
from sklearn.feature_extraction import DictVectorizer
vec = DictVectorizer()
vec.fit_transform(train_data[selected_features]).toarray()

AttributeError: 'str' object has no attribute 'iteritems'

In [26]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(train_data[selected_features], train_data.actual_class)

ImportError: No module named sklearn.ensemble